This Notebook steps :
    
   * Create a Spark session
    
   * Import training images to proceed, as binary, in a Spark DataFrame
    
   * Labeling images, by fruit name, extracted from images path 
    
   * Enhance image by tweeking color, sharpness, contrast, brightness
    
   * Extract 2048 features array by tranfert learning, using Keras Resnet50 CNN
    
   * Stores path, label and feature array, partitionned by label, in parquet format file on S3 
    

In [1]:
print('Welcome to my EMR Notebook!')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1631354174370_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Welcome to my EMR Notebook!

# Install dependencies

In [2]:
sc.install_pypi_package('pandas==1.2.5')

sc.install_pypi_package('pillow')

sc.install_pypi_package('pyarrow==2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/e6/0a/90da8840e044c329a0271fb0244ff40a68a2615bc360c296a3dc5e326ab6/pandas-1.2.5-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/d6/28/827b9cac687e086110eb133ab7e4f36ab4b35a1e1654c6329840ce045354/Pillow-8.3.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl

  Using cached https://files.pythonhosted.org/packages/c8/58/d07e7ee8b0cffe509f9e5a3742e09636a4a58b2113d193166615b934846f/pyarrow-2.0.0-cp37-cp37m-manylinux1_x86_64.whl

# Imports

In [3]:
import pandas as pd

from PIL import Image

from PIL import ImageEnhance

import numpy as np

import io

import os

from pyspark.sql import SparkSession

from pyspark.sql.functions import col, pandas_udf, udf, PandasUDFType, size

from pyspark.sql import types 

from pyspark import SparkContext, SparkConf

import tensorflow as tf

from tensorflow.keras.models import Model

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

from tensorflow.keras.preprocessing.image import img_to_array

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Constants

LOAD_PATH = 's3a://fruits-images-to-proceed/Training/'

SAVE_PATH = 's3a://fruits-images-proceded/Training_featured.parquet'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Enable pyArrow

In [5]:
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', 'true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Load images from storage

In [6]:
# Read jpg images as binary, recursively
images =(spark
         .read
         .format('binaryFile')
         .option('pathGlobFilter', '*.jpg')
         .option('recursiveFileLookup', 'true')
         .load(LOAD_PATH)
        )

images.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)

In [7]:
# Total number of images
totalMunber = images.count()
print('Total number of images in train set {}'.format(totalMunber))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total number of images in train set 67692

## Retrieve labels from image path

In [8]:
# Offset of starting image name
label_offset = len(LOAD_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#Get only fruit name from path

col_label = udf(lambda s : extract_label(s), types.StringType())

def extract_label(s):
    last = s[label_offset :]
    return last[:last.rfind('/')]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Create label column
images = images.withColumn('label',col_label(images.path))
images.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

In [11]:
#Get only fruit name from path
images.select('label').show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|label         |
+--------------+
|Raspberry     |
|Raspberry     |
|Pineapple Mini|
|Raspberry     |
|Raspberry     |
|Raspberry     |
|Pineapple Mini|
|Raspberry     |
|Raspberry     |
|Raspberry     |
|Raspberry     |
|Raspberry     |
|Raspberry     |
|Raspberry     |
|Raspberry     |
|Raspberry     |
|Raspberry     |
|Raspberry     |
|Raspberry     |
|Raspberry     |
+--------------+
only showing top 20 rows

In [12]:
# By label count
print('By label images count :')
images.groupBy('label').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

By label images count :
+-----------------+-----+
|            label|count|
+-----------------+-----+
|       Clementine|  490|
|     Cantaloupe 1|  492|
|     Pear Forelle|  702|
|           Orange|  479|
|      Onion White|  438|
|    Pepper Orange|  702|
|              Fig|  702|
| Strawberry Wedge|  738|
|  Grapefruit Pink|  490|
|         Beetroot|  450|
|      Cauliflower|  702|
|     Potato White|  450|
|Tomato Cherry Red|  492|
|    Grape White 4|  471|
|       Grape Blue|  984|
|     Cantaloupe 2|  492|
|    Passion Fruit|  490|
|        Mango Red|  426|
|       Nut Forest|  654|
|    Grape White 3|  492|
+-----------------+-----+
only showing top 20 rows

# Images enhancement

In [13]:
# Enhance image
def enhance(img,
            color = 1.25,
            sharpness = 4.5,
            contrast = 1.25,
            brigthness= 1.5):
    colorEnhancer = ImageEnhance.Color(img)
    img = colorEnhancer.enhance(color)
    
    sharpnessEnhancer = ImageEnhance.Sharpness(img)
    sharpnessEnhancer.enhance(sharpness)
    
    contrastEnhancer = ImageEnhance.Contrast(img)
    contrastEnhancer.enhance(contrast)
    
    brigthnessEnhancer = ImageEnhance.Brightness(img)
    brigthnessEnhancer.enhance(brigthness)
    
    return img

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Transfert learning (Resnet50)

In [14]:
def model_fn():
    '''
    Returns a ResNet50 model with top layer removed
    '''
    # imports Resnet
    resnet_full = ResNet50()
    # Create model excluding last classification layer
    model = Model(inputs = resnet_full.inputs,
                  outputs = resnet_full.layers[-2].output)
   
    return model

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
def preprocess(content):
    '''
    Preprocesses raw image bytes for prediction.
    '''
    # load raw image from dataframe and resize it to ResNet specifications
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    # Enhance image
    img = enhance(img)
    # image to Tensor array
    arr = img_to_array(img)
    # return ResNet50 preprocessed image
    return preprocess_input(arr)


def featurize_series(model, content_series):
    '''
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    '''
    #   input = np.stack(content_series.map(preprocess))
    input = tf.convert_to_tensor(np.stack(content_series.map(preprocess)), dtype=tf.float32)
    # features from image
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    # return features vector
    return pd.Series(output)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
from typing import Iterator

@pandas_udf('array<float>')
def featurize_udf(content_series_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
  '''
  This method is a Scalar Iterator pandas UDF wrapping our featurization function.
  The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
  
  :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
  ''' 
  # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
  # for multiple data batches.  This amortizes the overhead of loading big models.
  model = model_fn()
  for content_series in content_series_iter:
    yield featurize_series(model, content_series)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Apply featurization to the DataFrame of images

In [17]:
# Avoiding Out Of Memory (OOM) errors by reducing the Arrow batch size
spark.conf.set('spark.sql.execution.arrow.maxRecordsPerBatch', '512')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Transfert learning 
images = images.withColumn('features', featurize_udf(images.content))
images.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)

   # Save to storage

In [19]:
# Save Spark DataFrame, partitionned by label, in S3 Bucket
(images
 .select('path','label','features')
 .write
 .partitionBy('label')
 .mode('overwrite')
 .parquet(SAVE_PATH)
) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# End Spark session

In [20]:
spark.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…